In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.__version__

# scroll performance analysis

## define helper functions

In [ ]:
def label_string(s: str):
    return f"{s[:5]}...{s[-5:]}"

def get_suffix(s: str):
    return s.split("_")[-1] #.rstrip("0")

def vertical_line(df, g, value: int, color):
    for ax, col_name in zip(g.axes.flat, df["display_name"].unique()):
        subset = df[df["display_name"] == col_name]
        indices = subset[subset["time"] == value]["group_index"].values
        for index in indices:
            ax.axvline(x=index, color=color, linestyle="--", lw=1)

def lineplot_group(df, col_wrap, height, ylim):
    g = sns.FacetGrid(df[df["time"] >= 0], col="display_name", col_wrap=col_wrap, height=height, sharex=False, sharey=True)
    g.map(sns.lineplot, "group_index", "time")
    g.set(ylim=ylim)

    vertical_line(df, g, -10, "#AA00AA44") # data loading
    vertical_line(df, g, -20, "#00AAAAAA") # scroll event
   
    g.set_axis_labels("Frame", "Zeit in ms")
    g.set_titles("Renderzeit {col_name}")
    g.tight_layout()
    plt.show()

def filter(df, *patterns):
    condition = df["name"].str.contains("|".join(patterns))
    return df[condition]



# outlier removal 

range = 0.01
def remove_outliers_iqr(group):
    q1 = group["time"].quantile(range)
    q3 = group["time"].quantile(1-range)
    iqr = q3 - q1
    lower_bound = q1 - 3 * iqr
    upper_bound = q3 + 3 * iqr
    return group[(group["time"] >= lower_bound) & (group["time"] <= upper_bound) | (group["time"] < 0)]

## load raw measurements

In [ ]:
experiments = map(
lambda name: pd.read_csv(name),
[
    # "ios/logs/csv/2023-11-09_scrollBenchmark.csv",
    #"ios/logs/csv/2023-11-10_scrollBenchmarkSendMarkers.csv",
    #"android/logs/csv/2023-11-10_scrollInfiniteFlutterPC.csv", # infinite scroll without base64 image
    "android/logs/csv/2023-11-30_scroll.csv", # final scroll
    "ios/logs/csv/2023-12-01_scroll.csv", # final scroll
    #"ios/logs/csv/2023-11-10_scrollComplete.csv", # every scroll test scroll without base64 image
    #"ios/logs/csv/2023-11-13_scrollManual.csv", # infinite scroll (manual scroll) without base64 image
])

df = pd.concat(experiments, ignore_index=True)
#df.name = df.name.map(get_suffix)
df = df.groupby("name").apply(remove_outliers_iqr).reset_index(drop=True)
df.sort_values("name")
df

df.style.format({
    "time": "{:,d}",
})

pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [ ]:
analysis_df = df[df["time"] > 0]
analysis_df.groupby(["name"]).describe()

In [ ]:
#plot_df = df[(df["time"] <= 12) & ((df["time"] > 10) | (df["time"] < 0))]
plot_df = df
plot_df["group_index"] = plot_df.groupby("name").cumcount()

group_to_display_name = {
    "screenUpdate_T1a": "T1a: Android - Scroll", 
    "screenUpdate_T1b": "T1b: Android - InfiniteScroll", 
    "screenUpdate_T2a": "T2a: iOS - Scroll", 
    "screenUpdate_T2b": "T2b: iOS - InfiniteScroll", 
    "screenUpdate_T3a": "T3a: Android - Flutter Scroll", 
    "screenUpdate_T3b": "T3b: Android - Flutter InfiniteScroll", 
    "screenUpdate_T4a": "T4a: iOS - Flutter Scroll", 
    "screenUpdate_T4b": "T4b: iOS - Flutter InfiniteScroll", 
    "screenUpdate_T5a": "T5a: Android - Flutter Scroll Channel", 
    "screenUpdate_T5b": "T5b: Android - Flutter InfiniteScroll Channel", 
    "screenUpdate_T6a": "T6a: iOS - Flutter Scroll Channel", 
    "screenUpdate_T6b": "T6b: iOS - Flutter InfiniteScroll Channel", 
    }
plot_df["display_name"] = plot_df["name"].map(group_to_display_name)


lineplot_group(filter(plot_df, "T1a", "T3a","T5a"), 3, 5, (10, 25))

lineplot_group(filter(plot_df, "T1b", "T3b","T5b"), 3, 5, (10, 25))

lineplot_group(filter(plot_df, "T2a", "T4a","T6a"), 3, 5, (5, 30))

lineplot_group(filter(plot_df, "T2b", "T4b","T6b"), 3, 5, (5, 30))